### OpenAIEmbeddings
* 문서의 내용을 수치적인 벡터로 변환하는 과정
* 문서의 의미를 수치화, 모델들이 문맥적 정보를 이해 할 수 있게 한다.
* 벡터화된 문서들은 문서 분류, 감정 분석, 문서 간 유사도 계산에 사용 될 수 있다.

지원되는 모델 목록

| MODEL                  | PAGES PER DOLLAR | PERFORMANCE ON MTEB EVAL | MAX INPUT |
|------------------------|------------------|---------------------------|-----------|
| text-embedding-3-small | 62,500           | 62.3%                     | 8191      |
| text-embedding-3-large | 9,615            | 64.6%                     | 8191      |
| text-embedding-ada-002 | 12,500           | 61.0%                     | 8191      |

In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

In [3]:
text = "임베딩 테스트 위한 샘플 문장"

**쿼리 임베딩**

embeddings.embed_query(text)
* 주어진 텍스트를 임베딩 벡터로 변환하는 함수
* 텍스트를 벡터 공간에 매핑해, 의미적으로 유사한 텍스트를 찾거나 유사도를 계산할 때 사용

In [4]:
query_result = embeddings.embed_query(text)

In [5]:
query_result[:5]

[-0.00781757291406393,
 0.034758131951093674,
 0.031727321445941925,
 -0.0226589348167181,
 0.003623144468292594]

**Document 임베딩**

embeddings.embed_documents()

* 함수를 사용하여 텍스트 문서를 임베딩
* [text]를 인자로 전달해 단일 문서를 리스트 형태로 임베딩 함수에 전달

In [ ]:
doc_result = embeddings.embed_documents(
    [text, text, text, text]    # 네 장 짜리 문서
)

In [7]:
len(doc_result)

4

In [8]:
doc_result[0][:5]

[-0.00781757291406393,
 0.034758131951093674,
 0.031727321445941925,
 -0.0226589348167181,
 0.003623144468292594]

### 차원지정
* text-embedding-3-small 모델 같은 경우 1536차원의 임베딩을 반환

In [9]:
len(doc_result[0])

1536

In [11]:
# 차원이 1024인 text-embedding-3-small 모델 생성
embeddings_1024 = OpenAIEmbeddings(model="text-embedding-3-small", dimensions=1024)

In [12]:
doc_result_1024 = embeddings_1024.embed_documents([text,text,text,text])
len(doc_result_1024[0])

1024

### 유사도 계산

In [13]:
sentence1 = "안녕하세요? 반갑습니다."
sentence2 = "안녕하세요? 반갑습니다!"
sentence3 = "안녕하세요? 반가워요."
sentence4 = "Hi, nice to meet you."
sentence5 = "I like pizza."

### 코사인 유사도
-1 ~ 1 사이의 값 반환

**의미**
* 1에 가까울 수록 더 유사
* 0은 관계없음
* -1은 반대

In [16]:
!pip install -q scikit-learn

In [20]:
from sklearn.metrics.pairwise import cosine_similarity

sentences = [sentence1, sentence2, sentence3, sentence4, sentence5]
embedded_sentences = embeddings_1024.embed_documents(sentences)

In [19]:
def similarity(a,b):
    return cosine_similarity([a],[b])[0][0]

In [21]:
for i, sentence in enumerate(embedded_sentences):
    for j, other_sentence in enumerate(embedded_sentences):
        if i < j:
            print(
                f"[유사도 {similarity(sentence, other_sentence):.4f}] {sentences[i]} \t <=====> \t {sentences[j]}"
            )

[유사도 0.9644] 안녕하세요? 반갑습니다. 	 <=====> 	 안녕하세요? 반갑습니다!
[유사도 0.8469] 안녕하세요? 반갑습니다. 	 <=====> 	 안녕하세요? 반가워요.
[유사도 0.5042] 안녕하세요? 반갑습니다. 	 <=====> 	 Hi, nice to meet you.
[유사도 0.1016] 안녕하세요? 반갑습니다. 	 <=====> 	 I like pizza.
[유사도 0.8287] 안녕하세요? 반갑습니다! 	 <=====> 	 안녕하세요? 반가워요.
[유사도 0.4790] 안녕하세요? 반갑습니다! 	 <=====> 	 Hi, nice to meet you.
[유사도 0.0949] 안녕하세요? 반갑습니다! 	 <=====> 	 I like pizza.
[유사도 0.4784] 안녕하세요? 반가워요. 	 <=====> 	 Hi, nice to meet you.
[유사도 0.1123] 안녕하세요? 반가워요. 	 <=====> 	 I like pizza.
[유사도 0.2043] Hi, nice to meet you. 	 <=====> 	 I like pizza.
